In [1]:
!pip install crewai crewai_tools langchain_community sentence-transformers langchain-groq langchain_huggingface --quiet

In [2]:
!pip install langchain

In [1]:
import os
#os.environ['GROQ_API_KEY']='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'
os.environ['TAVILY_API_KEY']='tvly-z66T6o4dug8blrXv5eaZ4Ronsd3cVMZy___Q'


In [2]:
from langchain.agents import tool
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Task,Process
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI


In [3]:

"""
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    #model_name="llama3-8b-8192",
    model_name="llama-3.2-3b-preview",
    temperature=0.1,
    max_tokens=1000,

)
"""
# OR Use Ollama to run Locally
from langchain_community.llms import Ollama
llm = Ollama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)


In [4]:
#!pip install tf-keras

In [5]:
"""
rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

"""

rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3.2:latest",
                base_url="http://localhost:11434"
            ),
        ),
        embedder=dict(
                provider="huggingface", # or openai, ollama, ...
                config=dict(
                    model="BAAI/bge-small-en-v1.5",
                    #task_type="retrieval_document",
                    # title="Embeddings",
                ),
            ),
    )
)


In [6]:
question = "How did self-attention mechanism evolve in large language models in pdf?"
rag_tool.run(question)

Using Tool: Search a PDF's content


'Relevant Content:\nThis makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To the best of our knowledge, howev

In [7]:
@tool
def simple_router_tool(question : str) -> str:
  """Routes queries to the appropriate resources"""
  if "pdf" in question.lower():
    return "vectorstore"
  else:
    return "websearch"



In [8]:
test_input = {"question": "What does the  say about machine learning in pdf?"}
simple_router_tool.run(test_input)

'vectorstore'

In [10]:
#Agent 1

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 1
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[simple_router_tool],
)

In [11]:
crew1 = Crew(
    agents = [Router_Agent],
    tasks = [router_task],
    verbose= True,
)
test_input = {"question": "What does the  say about machine learning in pdf?"}
result1 = crew1.kickoff(inputs=test_input)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question What does the  say about machine learning in pdf?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Action: simple_router_tool
Action Input: {"question": "What does the  say about machine learning in pdf?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore




In [12]:
print(result1)

vectorstore


In [ ]:
web_search_tool = TavilySearchResults()
question = "How did self-attention mechanism evolve in large language models in pdf?"
web_search_tool(question)

/opt/anaconda3/envs/dev_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'url': 'https://glasp.co/hatch/7hz3t43jrtii5if6/p/pz1mQPOVZVdSiHasrCvK',
  'content': 'The self-attention mechanism is a fundamental building block of large language models like Transformers. It enables the model to capture the dependencies between different words in a sentence. The mechanism assigns weights to each word in the input sequence based on its relevance to other words. These weights are then used to compute a weighted'},
 {'url': 'https://ieeexplore.ieee.org/document/10245906',
  'content': 'Transformers, originally introduced for machine translation, and built upon the Self-Attention mechanism, have undergone a remarkable evolution, establishing themselves as the bedrock of large language models (LLMs). Their unparalleled capacity to model intricate relationships and capture extensive dependencies within sequences has propelled their prominence. This article, presented in a'},
 {'url': 'https://medium.com/@isaakmwangi2018/intro-to-differential-transformers-a-new-attentio

In [13]:
#Agent 2
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)




#Task 2
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
    #tools=[rag_tool, web_search_tool],
)

In [ ]:
# Crew for the System
simple_rag_crew = Crew(
    agents = [Router_Agent,Retriever_Agent],
    tasks = [router_task,retriever_task],
    verbose= True,
)


test_input = {"question" :  "How U-Net work?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result1 = simple_rag_crew.kickoff(inputs=test_input)

2024-11-22 00:07:20,884 - 8547711040 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How work U-Net?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Action: simple_router_tool
Action Input: {"question": "How work U-Net?"} 

websearch

Thought: I should check if the question is related to Retrieval-Augmented Generation, which might require a vectorstore search.
Action: simple_router_tool
Action Input: {"question": "How work U-Net?"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I should check if the question is related to Retrieval-Augmented Generation, which might require a vectorstore search.
Action: simple_router_tool
Action 

In [15]:
result1

"The self-attention mechanism in large language models has undergone significant evolution since its introduction. Initially, self-attention was used in the Transformer model (Vaswani et al., 2017) to enable parallelization of attention operations and improve efficiency. However, this early version had limitations, such as requiring a fixed window size for attention and being computationally expensive.\n\nLater, researchers introduced the concept of multi-head self-attention (Hecht-Mandelberg et al., 2020), which allows multiple attention heads to be applied simultaneously, improving the model's ability to capture different aspects of input sequences. This modification also enabled parallelization of attention operations across different attention heads.\n\nFurthermore, the development of layer normalization (Sorrentino et al., 2018) and positional encoding (Vaswani et al., 2017) has further improved the performance and efficiency of self-attention mechanisms in large language models. 